In [ ]:
import os
import random

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2

from tqdm import tqdm
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv2D, Dropout, Dense, Flatten, Input
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
# Make sure GPU is enabled
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
DIMS = (160,80)
EPOCHS = 200
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
IMG_DIR = 'data/'
SAVE_DIR = 'saved_models/'

In [ ]:
# UPDATE PATH HERE
%cd /content/drive/MyDrive/8voSemestre/CE903/lane_detection/

/content/drive/MyDrive/8voSemestre/CE903/lane_detection


In [ ]:
df = pd.read_csv('_controls.csv')
df.head()

,id,steer,throttle,brake
0,1,0.000627,0.379371,0.0
1,2,-0.037645,0.700000,0.0
2,3,-0.055207,0.700000,0.0
3,4,-0.001549,0.634152,0.0
4,5,-0.000116,0.648871,0.0


In [ ]:
lane_detector = load_model('full_CNN_model.h5')

In [ ]:
X = []
y = []

for file in tqdm(os.listdir(IMG_DIR)):
  id = int(file.split('.')[0])
  img = cv2.imread(IMG_DIR + file)

  # Prepare image for lane detection model
  img = cv2.resize(img, DIMS)
  img = np.array(img)
  img = img[None,:,:,:]

  # Generate lane mask and concatenate to original
  lane = lane_detector.predict(img)[0] * 255
  enhanced_img = np.concatenate((img[0], lane), axis=2)

  # Add image and labels to training data
  X.append(enhanced_img)
  y.append(df[df['id']==id].drop(columns=['id']).to_numpy()[0])

X = np.array(X)
y = np.array(y)

100%|██████████| 999/999 [01:26<00:00, 11.54it/s]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [ ]:
inputs = Input(shape=DIMS[::-1]+(4,))
x = Conv2D(16, 9, strides=(4,4), activation='relu', padding='same')(inputs)
x = Conv2D(32, 5, strides=(2,2), activation='relu', padding='same')(x)
x = Conv2D(64, 5, strides=(2,2), activation='relu', padding='same')(x)
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(3)(x)

model = Model(inputs, outputs, name='car_control')
model.compile(optimizer=Adam(lr=LEARNING_RATE),
              loss="mse")

model.summary()

Model: "car_control"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 80, 160, 4)]      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 20, 40, 16)        5200      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 20, 32)        12832     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 5, 10, 64)         51264     
_________________________________________________________________
flatten_2 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 3200)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss',
                               restore_best_weights=True,
                               patience=5)
  
checkpoint = ModelCheckpoint(SAVE_DIR+model.name)

history = model.fit(X_train, y_train,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_data=(X_test, y_test),
                    callbacks = [early_stopping, checkpoint])

Epoch 1/100
22/22 [==============================] - 0s 17ms/step - loss: 0.0479 - val_loss: 0.0412
INFO:tensorflow:Assets written to: saved_models/car_control/assets
Epoch 2/100
22/22 [==============================] - 0s 15ms/step - loss: 0.0493 - val_loss: 0.0435
INFO:tensorflow:Assets written to: saved_models/car_control/assets
Epoch 3/100
22/22 [==============================] - 0s 15ms/step - loss: 0.0509 - val_loss: 0.0402
INFO:tensorflow:Assets written to: saved_models/car_control/assets
Epoch 4/100
22/22 [==============================] - 0s 15ms/step - loss: 0.0488 - val_loss: 0.0434
INFO:tensorflow:Assets written to: saved_models/car_control/assets
Epoch 5/100
22/22 [==============================] - 0s 14ms/step - loss: 0.0491 - val_loss: 0.0420
INFO:tensorflow:Assets written to: saved_models/car_control/assets
Epoch 6/100
22/22 [==============================] - 0s 15ms/step - loss: 0.0467 - val_loss: 0.0448
INFO:tensorflow:Assets written to: saved_models/car_control/asset